<img src='https://upload.wikimedia.org/wikipedia/commons/thumb/a/aa/Logo_DuocUC.svg/2560px-Logo_DuocUC.svg.png' width=50%, height=20%>

# Árboles de decisión

Los árboles de decisión son métodos de aprendizaje de máquinas muy utilizados debido a su buen rendimiento y su explicabilidad. Un árbol de decisión no siempre tiene buen rendimiento, por lo que se proponen técnicas de ensamble de árboles de decisión aislados para generar un estimador más robusto que el estimador único por el que está conformado el ensamble.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn.tree # Árboles de decisión
import sklearn.ensemble # Ensambles de modelos
import sklearn.model_selection

Utilizaremos un conjunto de datos de predicción de diabetes dado una serie de parámetros fisiológicos de un paciente.

In [ ]:
diabetes = pd.read_csv("data/diabetes.csv")

In [ ]:
diabetes

Para ilustrar la construcción de un árbol de decisión utilizaremos sólo un par de variables del conjunto de datos.

In [ ]:
sns.scatterplot(
    data = diabetes,
    x = "Glucose",
    y = "BMI",
    hue = "Outcome",
    alpha = 0.5
)

# Árbol de decisión

Para la construcción de un árbol de decisión, primero debemos definir una métrica a optimizar. En el caso de los árboles de decisión, una métrica que podemos optimizar es la entropía. En donde a medida que vamos tomando decisiones buscamos ganar información o disminuir la entropía.

La definición de entropía que utilizaremos es la siguiente:

$$
H(X)=- \sum_{i}p(x_i) \log_2 p(x_i)
$$

Donde $X$ es el conjunto de etiquetas.

In [ ]:
def entropy(data):
    classes = np.unique(data)
    entropies = []
    for c in classes:
        p = sum(data == c) / len(data)
        current_entropy = p * np.log2(p)
        entropies.append(current_entropy)
    return -1 * sum(entropies)

Para comenzar, podemos estimar la entropía total del conjunto de datos

In [ ]:
entropy(diabetes.Outcome)

Ahora, debemos tomar una decisión, con la cual buscamos disminuir la entropía de cada uno de los subconjuntos obtenidos posterior a la decisión.

In [ ]:
entropy(diabetes.Outcome[diabetes.Glucose >= 175])

In [ ]:
entropy(diabetes.Outcome[diabetes.Glucose < 175])

Para estimar la entropía general de la decisión que tomamos debemos calcular un promedio ponderado de cada una de las entropías de cada subconjunto de datos.

In [ ]:
def weighted_entropy(data, feature, label, threshold):
    indices = data[feature] >= threshold
    entropy_0 = entropy(data[label][indices])
    entropy_1 = entropy(data[label][~indices])
    return entropy_0 * (sum(indices) / len(indices)) + entropy_1 * (sum(~indices) / len(indices))


In [ ]:
weighted_entropy(diabetes, "Glucose", "Outcome", 175)

Para determinar la mejor variable y el mejor umbral para la decisión del nodo del árbol debemos optimizar la ganancia de información de la decisión con la siguiente definición:

$$
IG(Y,X) = E(Y) - E(Y|X)
$$

Donde a la entropía total del conjunto de datos $E(Y)$ le restamos la entropía promedio $E(Y|X)$ de la decisión $Y$.

In [ ]:
entropy(diabetes.Outcome) - weighted_entropy(diabetes, "Glucose", "Outcome", 175)

Calculamos las entropías asociadas a un barrido de umbrales en un par de variables.

In [ ]:
glucose_information_gain_sweep = []
glucose_thresholds = np.linspace(diabetes.Glucose.min(), diabetes.Glucose.max(), 100)
for threshold in glucose_thresholds:
    glucose_information_gain_sweep.append(entropy(diabetes.Outcome) - weighted_entropy(diabetes, "Glucose", "Outcome", threshold))

In [ ]:
sns.lineplot(
    x = glucose_thresholds,
    y = glucose_information_gain_sweep
)

In [ ]:
bmi_information_gain_sweep = []
bmi_thresholds = np.linspace(diabetes.BMI.min(), diabetes.BMI.max(), 100)
for threshold in bmi_thresholds:
    bmi_information_gain_sweep.append(entropy(diabetes.Outcome) - weighted_entropy(diabetes, "BMI", "Outcome", threshold))

In [ ]:
sns.lineplot(
    x = bmi_thresholds,
    y = bmi_information_gain_sweep
)

In [ ]:
fig, axs = plt.subplots(nrows=1,ncols=2, sharey=True)
sns.lineplot(
    x = glucose_thresholds,
    y = glucose_information_gain_sweep,
    ax = axs[0]
)
sns.lineplot(
    x = bmi_thresholds,
    y = bmi_information_gain_sweep,
    ax = axs[1]
)
axs[0].set_title("glucose")
axs[1].set_title("bmi")
axs[0].set_ylabel("information_gain")
axs[0].set_xlabel("threshold")
axs[1].set_xlabel("threshold")

Seleccionamos el umbral que nos aporta la mayor ganancia de información.

In [ ]:
optimal_glucose_threshold = glucose_thresholds[np.argmax(glucose_information_gain_sweep)]
optimal_glucose_threshold

124.62626262626262

In [ ]:
entropy(diabetes.Outcome[diabetes.Glucose >= optimal_glucose_threshold])

0.9773203829731114

In [ ]:
entropy(diabetes.Outcome[diabetes.Glucose < optimal_glucose_threshold])

0.6930190480473644

Utilizamos la implementación del árbol de decisión de sklearn para calcular la misma decisión.

In [ ]:
single_decision = sklearn.tree.DecisionTreeClassifier(max_depth=1, criterion = "entropy")
single_decision = single_decision.fit(diabetes[["Glucose", "BMI"]], diabetes.Outcome)

In [ ]:
sklearn.tree.plot_tree( # Función que nos permite visualizar el árbol de decisión ajustado.
    single_decision, # Objeto de nuestro árbol de decisión entrenado.
    feature_names = ["Glucose", "BMI"], # Nombres de las variables utilizadas para entrenar.
    class_names = ["healthy","sick"], # Nombre de las clases que estamos prediciendo.
    label = "all", # Etiquetamos todas características de cada nodo.
    proportion = True, # Visualizamos las proporciones de datos en cada nodo de decisión,
    filled=True, # Coloreamos los nodos
    fontsize=11, # Establecemos el tamaño de la letra del texto dentro de cada nodo.
)
plt.show()

Preparamos el conjunto de datos para poder ajustar un árbol de decisión de mayor profundidad.

In [ ]:
diabetes_features = diabetes.iloc[:,:-1]
diabetes_label = diabetes.Outcome

In [ ]:
(
    diabetes_features_train, 
    diabetes_features_test, 
    diabetes_label_train, 
    diabetes_label_test
) = sklearn.model_selection.train_test_split(
    diabetes_features, 
    diabetes_label, 
    test_size=0.33, 
    random_state=11
)

In [ ]:
tree = sklearn.tree.DecisionTreeClassifier( # Instanciamos nuestro árbol de decisión.
    max_depth=3, # Forzamos que nuestro árbol sólo tenga 3 niveles de profundidad.
    criterion = "entropy"
    )
tree.fit( # Ajustamos nuestro árbol de decisión.
    diabetes_features_train,
    diabetes_label_train
)

Calculamos el rendimiento de nuestro árbol de decisión.

In [ ]:
print(sklearn.metrics.classification_report(
    diabetes_label_test,
    tree.predict(diabetes_features_test)
))

Visualizamos el árbol de decisión.

In [ ]:
plt.figure(figsize = (20,10))
sklearn.tree.plot_tree( # Función que nos permite visualizar el árbol de decisión ajustado.
    tree, # Objeto de nuestro árbol de decisión entrenado.
    feature_names = diabetes_features.columns, # Nombres de las variables utilizadas para entrenar.
    class_names = ["healthy","sick"], # Nombre de las clases que estamos prediciendo.
    label = "all", # Etiquetamos todas características de cada nodo.
    proportion = True, # Visualizamos las proporciones de datos en cada nodo de decisión,
    filled=True, # Coloreamos los nodos
    fontsize=11, # Establecemos el tamaño de la letra del texto dentro de cada nodo.
)
plt.show()

Un hiperparámetro que podemos ajustar en un árbol de decisión es la profundidad máxima. Visualizamos que tenemos un mejoramiento inicial del rendimiento al aumentar la profundidad, para después descender debido al sobreajuste.

In [ ]:
depths = range(1,10)
performances = []
for depth in depths:
    current_tree = sklearn.tree.DecisionTreeClassifier( # Instanciamos nuestro árbol de decisión.
        max_depth=depth, # Forzamos que nuestro árbol sólo tenga 3 niveles de profundidad.
        criterion = "entropy"
    )
    roc_auc = sklearn.model_selection.cross_val_score(
          current_tree,
          diabetes_features, 
          diabetes_label,
          scoring="roc_auc"
      ).mean()
    performances.append(roc_auc)

In [ ]:
plt.plot(
    depths,
    performances
)
plt.xlabel("max_width")
plt.ylabel("mean_roc_auc_score")
plt.show()